In [1]:
import xarray as xr
import numpy as np
import dask.array as da

In [2]:
coords = {
    "time": np.arange(100),
    "lat": np.arange(100),
    "lon": np.arange(100),
}

In [3]:
%%timeit
data_dasked = xr.DataArray(
    dims=["time", "lat", "lon"],
    coords=coords,
    data=da.empty((100, 100, 100)),
)

577 µs ± 7.76 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [4]:
%%timeit
data_numpy = xr.DataArray(
    dims=["time", "lat", "lon"],
    coords=coords,
    data=np.empty((100, 100, 100)),
)

210 µs ± 2.9 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


---

# Now we define a chunk


In [5]:
%%timeit
xr.DataArray(
    dims=["time", "lat", "lon"],
    coords=coords,
    data=da.empty((100, 100, 100)),
    name="dask",
    attrs={"units": "m"},
).chunk({"time": 10, "lat": 10, "lon": 10})

4.34 ms ± 48.7 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [6]:
%%timeit
xr.DataArray(
    dims=["time", "lat", "lon"],
    coords=coords,
    data=np.empty((100, 100, 100)),
    name="numpy",
    attrs={"units": "m"},
).chunk({"time": 10, "lat": 10, "lon": 10})

5.89 ms ± 150 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [7]:
%%timeit
xr.DataArray(
    dims=["time", "lat", "lon"],
    coords=coords,
    data=da.empty((100, 100, 100), chunks=(10, 10, 10)),
    name="dask_already_chunked",
    attrs={"units": "m"},
)

585 µs ± 5.35 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


## Results

We can see that the empty dask array that is already chunked at creation is the fastest to compute.


---


#### Here we want to see if the mirror creation is worth the time it takes to create the chunks.


In [13]:
data_chunked = xr.DataArray(
    dims=["time", "lat", "lon"],
    coords=coords,
    data=np.empty((100, 100, 100)),
    name="numpy",
    attrs={"units": "m"},
).chunk({"time": 10, "lat": 10, "lon": 10})

In [15]:
%%timeit
results = data_chunked.copy()
if results.chunks != (10, 10, 10):
    results = results.chunk({"time": 10, "lat": 10, "lon": 10})

254 µs ± 6.18 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [17]:
data = xr.DataArray(
    dims=["time", "lat", "lon"],
    coords=coords,
    data=np.empty((100, 100, 100)),
    name="numpy",
    attrs={"units": "m"},
)

In [18]:
%%timeit
results = data.copy()
if results.chunks != (10, 10, 10):
    results = results.chunk({"time": 10, "lat": 10, "lon": 10})

6.08 ms ± 111 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


Mirroring when chunking is not necessary is **twice as fast** than lazy creation.

But if we need to chunk the mirrored array, it is more than **10 times slower** than the lazy creation.

Only LazyTemplate supplies should be used.
